In [1]:
%load_ext autoreload
%autoreload 2

import tkp.db  # NOTE: this only works with the queryqueue branch of TraP
import logging

import tkp.db.api
from pandas import DataFrame

logging.basicConfig(level=logging.INFO)

### Database connection settings

In [2]:
engine = 'postgresql'
host = 'localhost'
port = 5433
user = 'gijs'
password = user
database = user
query_loglevel = logging.WARNING  # Set to INFO to see queries, otherwise WARNING

### connect to the database

In [3]:
logging.getLogger('sqlalchemy.engine').setLevel(query_loglevel)
db = tkp.db.Database(engine=engine, host=host, port=port,
                     user=user, password=password, database=database)
db.connect()
session = db.Session()

### Execute the query

In [8]:
transient_query = tkp.db.api.transients(session, dataset=tkp.db.model.Dataset(id=1494))
columns = [c['name'] for c in transient_query.column_descriptions]
df = DataFrame(transient_query.all(), columns=columns)
df

,id,ra,decl,wm_uncertainty_ew,wm_uncertainty_ns,xtrsrc,dataset_id,datapoints,v_int,eta_int,band,newsource,sigma_rms_max,sigma_rms_min,lightcurve_max,lightcurve_avg,lightcurve_median
0,32,1,1,1,1,311,1494,1,0,0,1507,32,None,None,0.01,0.01,0.01
1,31,1,1,1,1,301,1494,1,0,0,1506,31,None,None,0.01,0.01,0.01
